# Het enige verschil met #2/3 is de dubbel check functie

Om alle leden in een ketzer te mailen hebben we een aantal packages nodig. <br>
De meeste zitten standaard bij python, sommige niet: als je pandas of numpy niet hebt installeer die dan. <br>
Dat kan met: _pip install pandas_ en _pip install numpy_

In [40]:
# packages nodig voor mailen
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
import time
from IPython.core.display import HTML

# data management == heel handig
import pandas as pd
import numpy as np

We beginnen met het ophalen van ons excel, stop je excel in de zelfde map en voer de naam in waar staat _Leden van activiteit.xlsx_ . <br>
Zo lang het ketzer systeem het zelfde blijft hebben we altijd de kollomen 'Unnamed: 0', 'Uid', 'Lidnaam', 'Opmerking', 'Opmerking2', 'Lid sinds','Aangemeld door'. Deze behouden we maar gebruiken voor _'lidnaam'_ voor aanhef en _'Uid'_ voor mail.<br>
we slaan de eerste rij over, deze is altijd 'C.S.R. Delft - Beheer activiteiten', dat is de skiprows[0]

In [20]:
df_in = pd.read_excel("Leden van Lustrum gala.xlsx",skiprows=[0])
df_in.columns

Index(['Unnamed: 0', 'Uid', 'Lidnaam', 'Opmerking', 'Opmerking2', 'Lid sinds',
       'Aangemeld door'],
      dtype='object')

Ter controlle de excel zie je dan hieronder:

In [21]:
df_in

,Unnamed: 0,Uid,Lidnaam,Opmerking,Opmerking2,Lid sinds,Aangemeld door
0,NaN,x101,Am. Lid,NaN,NaN,2022-05-20 12:12,Jan Lid
1,NaN,x102,Ama. Lid,NaN,NaN,2022-05-20 12:12,Jannette Lid
2,NaN,x103,Am. Lid2,"Ama. Lid2 [0,1]",NaN,2022-05-20 12:12,Jan Lid
3,NaN,x104,Ama. Lid2,NaN,NaN,2022-05-20 12:12,Jannette Lid
4,NaN,x105,Am. Lid3,"Ama. Lid3 (0,1)",NaN,2022-05-20 12:12,Jan Lid


Vervolgens laten we de lege kolom weg (unnamed:0) en kunnen we leden mailen met Uid@csrdelft.nl (ja vet handig)

In [37]:
# hij spuit een error maar dat is geen probleem in principe dus dit fixt het :)
import warnings
warnings.filterwarnings("ignore")

# behouden de kollomen
df = df_in[['Uid','Lidnaam','Opmerking','Opmerking2']]
# voegen een mail kollom aan
df['Mail'] = df['Uid'].apply(lambda x: f'{x}@csrdelft.nl')

# en maak van uid de index:
if "Uid" in df.columns: # check om error te voorkomen
    df = df.set_index("Uid")
    
# df = df.loc[['x101']]
df

,Lidnaam,Opmerking,Opmerking2,Mail
Uid,,,,
x101,Am. Lid,NaN,NaN,x101@csrdelft.nl
x102,Ama. Lid,NaN,NaN,x102@csrdelft.nl
x103,Am. Lid2,"Ama. Lid2 [0,1]",NaN,x103@csrdelft.nl
x104,Ama. Lid2,NaN,NaN,x104@csrdelft.nl
x105,Am. Lid3,"Ama. Lid3 (0,1)",NaN,x105@csrdelft.nl


Wat vaak gebeurt is dat leden er dubbel in staan, je ziet dat janlid2 met jannette lid2 gaat, maar jannette heeft zich niet uitgekezt.<br>
Noot dat vaak [0,1] eten voorkeuren aangeven <br>
Met 300 leden naar een gala is dit wat lastig, daarom deze code: 

In [23]:
def check_double(df):
    lst = []
    if 'Lidnaam' in df.columns and 'Opmerking' in df.columns:
        # haal de df op waar iedereen ee ndat heeft
        df_to_check = df[['Lidnaam','Opmerking']].dropna()
        
        # maak lijsten van de aparte twee 
        met_dates = df_to_check.index.to_list()
        iedereen = df.index.to_list()
        
        # verschil er tussen
        zonder_date = list(set(iedereen).difference(set(met_dates)))
        
        # maak df van iedeereen zonder date:
        df_to_compare_to = df.loc[df.index.isin(zonder_date)]
        # hiervan maakt alleen de naam van de persoon die is ingekezt uit:
        namen_ingekezt = df_to_compare_to["Lidnaam"].to_list()
        
        # loop door alle namaen dates heen:  
        for row in df_to_check.index: 
            naam_date = str(df_to_check.loc[row,'Opmerking'])
#             print(naam_date)
            # alleen amicale namen:
            if naam_date[:2].upper() == "AM":
                if naam_date.find("[") == -1:
                    print(f"Pheut {row} met een () ipv []")
                    if naam_date.find("(") != -1:
                        naam_date_vinden = naam_date[:naam_date.find("(")]
                    else:
                        raise Warning(f"Geen eetwens bij {row}")
                else:
                    naam_date_vinden = naam_date[:naam_date.find("[")]
                if naam_date_vinden[-1] == " ":
                    naam_date_vinden = naam_date_vinden[:-1]
                if naam_date_vinden in namen_ingekezt:
                    print(naam_date_vinden, row, namen_ingekezt.index(naam_date_vinden))
                    lst.append(naam_date_vinden)
            else:
                pass
        if len(lst) == 0:
            print("Geen dubbele")
        else:
            print(lst)
        # 
    else:
        raise Warning("Columns invalid")

In [24]:
check_double(df)

Ama. Lid2 x103 2
Pheut x105 met een () ipv []
['Ama. Lid2']


In [38]:
df = df.loc[['x101']]

Stel je wilt een deel los mailen, dan kan je makkelijkst selecteren op lid nummer.<br>
Bij echte uids kan je gwn een nummer ipv string invoeren<br>
Haal de twee # weg om te zien hoe het werkt (dit doe ik al hier boven)

Dan het echte verzenden:

In [49]:
############# Dingen om aan te passen #############
# deze is heel belangrijk, zorg dat hij bij testen op FALSE staat, anders verstuur je perongelijk mailtjes
send = False
# pas je commissie mail aan
sender_email = "commissie@csrdelft.nl"

############# Instellingen #############
# wat server instellingen belangrijk om verbinding te maken met de server, inprincipe generiek 
port = 587
smtp_server = "smtp.knorrie.org"

# prompt om je wachtwoord in te voergen: 
if send:    
    password = input("Type your password and press enter:\n")

# bijhouden voor handigheid
count_kamers = 0
# voor alle rijen in de excel
for i in range(len(df)):
    # per rij nemen we die specifieke
    row = df.iloc[i]
    
    # check voor een lege rij
    if pd.isna(row['Opmerking']):
        ############# Deel wat anders is: #############
        # als de rij opmerkingen leeg is krijg je dus geen mail. 


        ############# Instellingen #############
        # maak een message opject aan 
        message = MIMEMultipart("alternative")

        # de verstuurder is de commissie
        message["From"] = sender_email

        # de ontavnger halen we uit de rij 
        receiver_email = row['Mail']
        message["To"] = receiver_email


        ############# Dingen om aan te passen #############
        #### gebruik bijv dit om de html opmaak te bouwen: https://html-online.com/editor/  of met de hand ####

        # geef de mail een title
        message["Subject"] = f"Voorbeeldmail naar alle leden in de activiteiten ketzer"
            # add main body
        # add main body
        body = f"""\
            <html>
              <body>
                    <p>Beste {row.Lidnaam}, <br></p>
                    <p>Het gala van 21 oktober komt er nogsteeds aan.</p>
                    <p> Om alles af te kunnen ronden horen wij graag voor 12 augustus 00:12 2022 wie u date is.<br>
                    <b>Dit is al over 10 dagen al!<br>
                    Dit klinkt misschien vroeg maar om de nodige voorbereidingen te kunnen treffen
                    hebben wij tijdig een overzicht nodig van de aanwezigen.<br>
                    Vooral ook omdat we graag de novieten verwelkomen op het gala. <br>
                    Schrijf dus vooral uw blauwe/roze brieven en zet u date er bij in de ketzer!</b>
                    </p>              


                    <p>Met vriendelijke groet,
                    <br> De 12e LustrumCie der C.S.R. Delft</p>

              </body>
            </html>

            """
        ############# Verzenden #############
        # je voegt dan de html toe
        message.attach(MIMEText(body, "html"))

        # je kan ook een foto toevoegen bijv. 
        picture = False
        if picture:
            with open('Postergala.jpeg','rb') as fin:
                img = MIMEImage(fin.read())
                img.add_header('Content-Disposition', 'inline; filename=poster.jpg')
                message.attach(img)


        # Als je aan het testen ben kan je printen aan zetten en dan zie je een voorbeeld
        printing = True

        # dit verstuurd het daadwerkelijk
        if send:
            context = ssl.create_default_context()
            with smtplib.SMTP(smtp_server, port) as server:
                server.ehlo()  # Can be omitted
                server.starttls(context=context)
                server.ehlo()  # Can be omitted
                server.login(sender_email, password)
                server.sendmail(sender_email, receiver_email, message.as_string())

            print(f"{i}: {receiver_email} sent")

        # als het een test is print hij het het bericht, moved to html out below
#         else:
#             
# #             if printing:
# #                 print(message)
# # #                 pass
    
        # als je deze er niet in doet stopt ie na 100 mails
        time.sleep(1)
print(f'{count_kamers} kamers')
HTML(body)

0 kamers


Hierboven een voorbeeld mail in tekst en dit heironder hoe het in gmail er uit ziet

![Hierboven een voorbeeld mail ](Voorbeeld3.png)